In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel(r"C:\Users\m.olshanskiy\Desktop\База декабрь\Новая папка\База 122.xlsx")

In [3]:
df['Дата обновления'] = pd.to_datetime(df['Дата обновления'], errors='coerce')


In [9]:
# df = df[df['Округ'].isin(['НАО', 'ТАО'])]

In [5]:
# Шаг 1 — исключаем квартиры от Фонда Реновации
df_filtered = df[df['Девелопер'] != 'Фонд реновации'].copy()

# Шаг 2 — добавляем числовой месяц
df_filtered['месяц'] = df_filtered['Дата обновления'].dt.month

# Шаг 3 — группировка по проекту и девелоперу + подсчёт квартир по месяцам
project_month_counts = (
    df_filtered
    .groupby(['Название проекта', 'Девелопер', 'месяц'])
    .size()
    .unstack(fill_value=0)
    .rename(columns={11: 'ноябрь', 10: 'октябрь'})
)
project_month_counts = project_month_counts.rename(columns={11: 'ноябрь', 10: 'октябрь'})

# Шаг 4 — фильтрация:
# - минимум 5 квартир в июне
# - июль не ноль
# - изменение >= 80% в любую сторону
filtered = project_month_counts[
    (project_month_counts['октябрь'] >= 1) &
    (project_month_counts['ноябрь'] >= 0) &
    (
        (project_month_counts['ноябрь'] >= project_month_counts['октябрь']) |
        (project_month_counts['ноябрь'] <= project_month_counts['октябрь'])
    )
]

# Шаг 5 — читаемый вывод
result = [
    f"{project} — октябрь: {row['октябрь']}, ноябрь: {row['ноябрь']}"
    for project, row in filtered.iterrows()
]

# Вывод
for line in result:
    print(line)

('1-й Донской', 'ФСК') — октябрь: 681, ноябрь: 685
('1-й Измайловский', 'ФСК') — октябрь: 262, ноябрь: 303
('1-й Ленинградский', 'ФСК') — октябрь: 108, ноябрь: 94
('1-й Лермонтовский', 'ФСК') — октябрь: 115, ноябрь: 96
('1-й Саларьевский', 'ФСК') — октябрь: 230, ноябрь: 230
('1-й Химкинский', 'ФСК') — октябрь: 337, ноябрь: 501
('1-й Шереметьевский', 'ФСК') — октябрь: 380, ноябрь: 420
('1-й Южный', 'ФСК') — октябрь: 477, ноябрь: 400
('1-й Ясеневский', 'ФСК') — октябрь: 282, ноябрь: 334
('1864 Резиденция', 'Сбер Капитал') — октябрь: 24, ноябрь: 23
('2-й Иртышский', 'ПИК') — октябрь: 221, ноябрь: 191
('31 Квартал', 'ПРОФИ-Инвест') — октябрь: 170, ноябрь: 144
('7 небо', 'ПРОФИ-Инвест') — октябрь: 174, ноябрь: 163
('8 Кленов', 'Сити 21 век') — октябрь: 177, ноябрь: 157
('AFI Park Воронцовский', 'AFI') — октябрь: 15, ноябрь: 6
('AHEAD', 'VOS’HOD') — октябрь: 22, ноябрь: 22
('ARTEL', 'Stenoy') — октябрь: 96, ноябрь: 97
('Adres', 'СЗ Т-ОТЕЛЬ') — октябрь: 59, ноябрь: 56
('Afi Tower', 'AFI') — о

In [16]:
filtered.head()

,месяц,сентябрь,октябрь,Разница
Название проекта,Девелопер,,,
1-й Донской,ФСК,736,681,55
1-й Измайловский,ФСК,347,262,85
1-й Ленинградский,ФСК,123,108,15
1-й Лермонтовский,ФСК,179,115,64
1-й Саларьевский,ФСК,227,230,-3


In [11]:
filtered['Разница'] = filtered['сентябрь'] - filtered['октябрь']

C:\Users\m.olshanskiy\AppData\Local\Temp\ipykernel_21932\3930492352.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['Разница'] = filtered['сентябрь'] - filtered['октябрь']


In [14]:
print(filtered['Разница'].sum())

10057


In [17]:
filtered.to_excel(r"C:\Users\m.olshanskiy\Desktop\разница.xlsx")